SZALKA ÁRON DATA SCIENCE kurzus beszámoló SZISZ 2023.

A beszámolóban filmkritikákat töltöttem le web-scraping segítségével, a kritikákat elemeztem és vizualizáltam, majd megpróbáltam olyan modellt építeni, ami megbecsüli, hogy az adott kritika melyik oldalon jelenhetett meg.

A beszámoló első része az adatok letöltése. Az imdb top 5 filmjéhez töltöttem le az össze user review-t az IMDB-ről, Rotten tomatoes-ról és a Letterboxd-ról (több, mint 5filmet szerettem volna eredetileg, de nagyon sok kritika volt már 5 filmhez is, és nagyon hosszúak voltak a letöltési idők). Ez így körülbelül 180 ezer kritia letöltését jelentette, amit megnehezített, hogy az IMDB és a Rotten Tomatoes oldalán csak Seleniummal lehetett elérni az összes kritikát, de ezek az oldalak néha elérhetetlenné váltak a próbálkozásaim után (feltételezem valami túlterhelésgátlóval védekeznek?), így többször is kellett próbálkozni minden filmnél. Mindegyik kritikához letöltöttem, hogy hány csillagot adott az adott kritika a filmre, ha értékelte. Ezeket az adatokat mind külön pickle-ökbe mentettem, és csatolom a beszámolóhoz. 

A dolgozat második részében az így szerzett adatokat beolvastam a picke-ökből és elrendeztem egy Pandas Dataframe-be, tisztítottam és vizualizáltam. Itt már kiderült néhány probléma, elsősorban, hogy az adatok között nics túl nagy szórás. Az értékelések szinte minden esetben 9-10 körül vannak, a kritikák hossza AZ IMDB esetében eltérő ugyan valamennyire, de a másik két oldalon nagyon hasonlóan rövidek. Ezenfelül az adatok sem oszlanak meg egyenlően az oldalak között, a Rotten Tomatoes oldalon sokkal több kritika jelent meg mint a többin. 

Végül a dolgozat utolsó részében megépítettem egy predikciós modellt. Sajnos sem ez, sem egyik korábbi próbálkozásom sem haladta meg nagyon sokkal az az arányt, amelyet a Rotten Tomatoes önmagában képvisel a reviewk számát tekintve. Amikor azt lecsökkentettem, akkor sem let jobb. Az a gyanúm, hogy a Rotten Tomatoes és a Letterboxd user review-i nagyon hasonlóak. Mindazonáltal a modell így is képes felülmúlni azt a józan paraszti logikát, hogy amennyiben egy kritika hosszú, akkor IMDB, egyébként Rotten Tomatoes, hiszen abból több van, mint Letterboxed-ból. Ezt a cofusion matrix is megerősíti, így kijelenthető, hogy volt értelme a modellnek.


In [1]:
#Importálás. A végső modellben nem használtam mindet az importáltak közül, de nem törötlem ki utólag azokat sem, hogy emlékezzek mikkel próbálkoztam. A kódban viszont csak a legjobb prediktív erejű modellt hagytam benne.

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.ensemble import RandomForestClassifier
from textblob import TextBlob
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.sparse as sp
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [ ]:
# Az IMDB toplista websrcapelése, és a címek segítségével mindhárom kritika oldal linklistájának megalkotása

url = "https://www.imdb.com/chart/top"
headers = {"Accept-Language": "en-US,en;q=0.9"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table", {"class": "chart full-width"})
rows = table.find_all("tr")

imdbtitlelist = []
imdblinklistelő = []

for row in rows[1:]:
    title_column = row.find("td", {"class": "titleColumn"})
    movie_title = title_column.a.text.strip()
    imdbtitlelist = imdbtitlelist + [movie_title]
    link_sor = row.find("a")
    link = link_sor["href"]
    imdblinklistelő = imdblinklistelő + [link]

imdblinklist = ["https://www.imdb.com" + title + "reviews?ref_=tt_urv" for title in imdblinklistelő]

letterboxdtitlelist = [title.replace(" ", "-") for title in imdbtitlelist]
letterboxdtitlelist = [title.lower() for title in letterboxdtitlelist]
letterboxdlinklist = ["https://letterboxd.com/film/" + title + "/reviews/" for title in letterboxdtitlelist]

rottentitlelist = [title.replace(" ", "_") for title in imdbtitlelist]
rottentitlelist = [title.lower() for title in rottentitlelist]
rottenlinklist = ["https://www.rottentomatoes.com/m/" + title + "/reviews?type=user" for title in rottentitlelist]

In [ ]:
#IMDB egy adott filmhez tartozó összes review letöltése Seleniummal.
#Sajnos loopolni nem sikerült, mert valószínűleg az IMDB ezt blokkolja (többször 404-et írt ki az ilyen próbálkozásaim után mikörben a többi honlap bejött).

path_to_chromedriver = 'I:/Eszkozok/Chromedriver/chromedriver.exe'  
service = Service(path_to_chromedriver)
driver = webdriver.Chrome(service=service)
driver.get("https://www.imdb.com/title/tt0050083/reviews?ref_=tt_urv")
wait = WebDriverWait(driver, 100)

while True:
    try:
        next_button = driver.find_element(By.XPATH, "//button[contains(@class, 'ipl-load-more__button') and @data-target-container='reviews-container' and @id='load-more-trigger']")
        driver.execute_script("arguments[0].click();", next_button)
        wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "lister-item-content")))
        
    except:
        break


html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
lista3 = []

reviews = soup.find_all("div", class_="lister-item mode-detail imdb-user-review collapsable")
        
for review in reviews:
    if review.find('span', class_='rating-other-user-rating') is not None:
        darab = review.find('span', class_='rating-other-user-rating').find('span').get_text()
    else:
        darab = "Na"
        
        
    if review.find("div", class_="text show-more__control") is not None:
        review_text = review.find("div", class_="text show-more__control").get_text().strip()
    elif review.find("div", class_="text show-more__control clickable") is not None:
        review_text = review.find("div", class_="text show-more__control clickable").get_text().strip()
    else:
        review_text = "Na"
    lista3 = lista3 + [[darab, review_text]]


In [75]:
#IMDB webscrape picke-be mentése

with open('12 Angry Men Reviews IMDB.pkl', 'wb') as file:
    pickle.dump(lista3, file)

In [ ]:
#Egy Rotten tomates filmhez az összes review letöltése Seleniummal

path_to_chromedriver = 'I:/Eszkozok/Chromedriver/chromedriver.exe'  
service = Service(path_to_chromedriver)
driver = webdriver.Chrome(service=service)
driver.get("https://www.rottentomatoes.com/m/1000013_12_angry_men/reviews?type=user")
lista = []

while True:
   
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    reviews = soup.find_all("div", class_="audience-review-row")
    
    for review in reviews:
        darab = 0
        darab = darab + len(review.find_all("span", class_="star-display__filled")) + 0.5*len(review.find_all("span", class_="star-display__half"))
        review_text = review.find("p", class_="audience-reviews__review js-review-text").get_text().strip()
        if [darab, review_text] not in lista:
            lista = lista + [[darab, review_text]]

    next_buttons = driver.find_elements(By.XPATH, "//div[@class='prev-next-paging__wrapper']//rt-button[contains(@class, 'next')]")
    if len(next_buttons) == 0:
        break

    next_button = next_buttons[0]
    driver.execute_script("arguments[0].click();", next_button)
    
driver.quit()

In [10]:
#Rotten Tomatoes webscrape mentése picke-be
with open('12 Angry Men Reviews Rotten Tomatoes.pkl', 'wb') as file:
    pickle.dump(lista, file)

In [ ]:
#Egy Letterboxd filmhez az összes review letöltése

numbers = [num for num in range(1, 1000)]
lista2 = []

url = letterboxdlinklist[4]

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


reviews = soup.find_all("li", class_="film-detail")

for review in reviews:
    div_element = review.find("div", class_="attribution-block")
    span_element = div_element.find("span")  
    class_name = span_element.get("class")
    if class_name[0] == "rating":
        darab =  int(class_name[-1].split("-")[1])
    else:
       darab = "Na"
    try:
        review_text = review.find("div", class_="body-text -prose collapsible-text").find("p").get_text().strip()
    except AttributeError:
        review_text = None
    lista2 = lista2 + [[darab, review_text]]


for i in numbers:

    url = letterboxdlinklist[4] + "page/" + str(i) + "/"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")


    reviews = soup.find_all("li", class_="film-detail")

    for review in reviews:
        div_element = review.find("div", class_="attribution-block")
        span_element = div_element.find("span")  
        class_name = span_element.get("class")
        if class_name[0] == "rating":
            darab =  int(class_name[-1].split("-")[1])
        else:
            darab = "Na"

        #review_text = review.find("div", class_="body-text -prose collapsible-text").find("p").get_text().strip()
        try:
            review_text = review.find("div", class_="body-text -prose collapsible-text").find("p").get_text().strip()
        except AttributeError:
            review_text = None

        lista2 = lista2 + [[darab, review_text]]

In [40]:
#Letterboxd webscrape mentése picke-be
with open('12 Angry Men Reviews Letterboxd.pkl', 'wb') as file:
    pickle.dump(lista2, file)

In [ ]:
#Mentett pickle-ök beolvasása

titles = ['The Shawshank Redemption',
 'The Godfather',
 'The Dark Knight',
 'The Godfather Part II',
 '12 Angry Men']
pages = ['IMDB', 'Rotten Tomatoes', 'Letterboxd']
data = []
for i in titles:
    for j in pages:
        data2 = []
        try:
            pickle_file_path = i + " Reviews " + j + ".pkl"
            with open(pickle_file_path, "rb") as file:
                data2 = pickle.load(file)
            for sublist in data2:
                sublist.append(i)
                sublist.append(j)
                
            data = data + data2
        except FileNotFoundError:
            print(f"File {pickle_file_path} not found.")
        except Exception as e:
            print(f"Error loading file {pickle_file_path}: {str(e)}")

In [ ]:
#A letöltött és összefésült review-k filterezése, a kész df előállítása

df_filterelőtt = pd.DataFrame(data)
column_names = ['Score', 'Review', 'Movie', 'Page']
df_filterelőtt.columns = column_names
df_filterelőtt = df_filterelőtt.dropna(subset=['Review'])
df_filterelőtt = df_filterelőtt[df_filterelőtt['Review'] != 'Na']
df_filterelőtt['Length'] = df_filterelőtt.Review.str.len()
df_filterelőtt['Wordcount'] = df_filterelőtt.Review.str.split().str.len()
df_filterelőtt['Score'] = df_filterelőtt.apply(lambda row: row['Score'] * 2 if row['Page'] == 'Rotten Tomatoes' else row['Score'], axis=1)
df_filterelőtt['Score'] = pd.to_numeric(df_filterelőtt['Score'], errors='coerce')
df = df_filterelőtt[df_filterelőtt['Review'] != 'This review may contain spoilers. I can handle the truth.']


z_scores = stats.zscore(df['Length'])
threshold = 2
threshold2 = 5
filtered_df = df[abs(z_scores) < threshold]
filtered_df2 = df[abs(z_scores) < threshold2]

In [ ]:
#Itt következnek a különböző plotok, de elküldöm külön is:

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
sns.boxplot(x='Page', y='Length', data=filtered_df, ax=axes[0])
axes[0].set_title('Filtered DataFrame 1')
sns.boxplot(x='Page', y='Length', data=filtered_df2, ax=axes[1])
axes[1].set_title('Filtered DataFrame 2')
plt.tight_layout()
plt.show()

In [ ]:
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Movie', y='Score', hue='Page', data=df)
plt.xticks(rotation=90)
plt.title('Box Plot of Movie Scores by Page')
plt.show()

In [ ]:
avg_lengths = df.groupby(['Movie', 'Page'])['Length'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='Movie', y='Length', hue='Page', data=avg_lengths)
plt.title('Average Length by Movie and Page')
plt.show()

In [ ]:
sns.countplot(x='Page', data=df)
plt.title('Bar Plot of Row Counts by Category')
plt.show()

In [ ]:
sns.barplot(x='Page', y='Score', data=df)
plt.title('Bar Plot with Mean of Values')
plt.show()

In [ ]:
#Az általam épített algoritmusok közül a legjobb accuray score-ral rendelkező modell.(legalábbis egyelőre)
#0.9398 valószínűséggel találja el a honlapot, igaz, a Rotten Tomatoes 80.2%-át adja a kritikáknak, ami növeli a magas találat esélyét.

df = df.dropna()  
features = df[['Movie', 'Score', 'Length', 'Review']]
target = df['Page']

text_preprocessor = TfidfVectorizer(preprocessor=WordNetLemmatizer().lemmatize)
categorical_features = ['Movie']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_preprocessor, 'Review'),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LinearSVC())
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#A teljes és a filmekre lebontott confusion mátrxok alátámasztják, hogy a modellnek volt értelme. Az IMDB-t nagyon pontosan kitalálja, ami nem is meglepő.
#Azonban a Letterboxd és a Rotten Tomatoes kritikák esetében is kevesebbszer lő mellé, mint amennyi a Letterboxd kritikák száma.
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



movies = df['Movie'].unique()
for movie in movies:
    movie_mask = X_test['Movie'] == movie
    movie_y_test = y_test[movie_mask]
    movie_y_pred = y_pred[movie_mask]

    cm = confusion_matrix(movie_y_test, movie_y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {movie}')
    plt.show()